In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.impute import SimpleImputer

# Cargar el dataset seleccionado
selected_features = pd.read_csv("selected_features_rfe_optimized_100.csv")

# Separar las características (X) y la variable objetivo (y)
X = selected_features.drop(columns=['label'])  # Características
y = selected_features['label']  # Variable objetivo

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# Parámetros para la búsqueda de hiperparámetros
param_grid = {
    'L2 (Ridge)': {
        'model': LogisticRegression(penalty='l2', solver='saga', max_iter=1000, multi_class='multinomial', random_state=42),
        'param_grid': {'C': [0.01, 0.1, 1, 10, 100]}
    },
    'L1 (Lasso)': {
        'model': LogisticRegression(penalty='l1', solver='saga', max_iter=1000, multi_class='multinomial', random_state=42),
        'param_grid': {'C': [0.01, 0.1, 1, 10, 100]}
    },
    'Elastic-Net': {
        'model': LogisticRegression(penalty='elasticnet', solver='saga', max_iter=1000, multi_class='multinomial', random_state=42),
        'param_grid': {'C': [0.01, 0.1, 1, 10, 100], 'l1_ratio': [0.1, 0.5, 0.9]}
    }
}

# Resultados de cada modelo
for reg_name, config in param_grid.items():
    print(f"\nOptimizando modelo con regularización: {reg_name}")
    model = config['model']
    param_grid = config['param_grid']

    # GridSearchCV para optimización
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1_weighted', cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    # Mejor modelo
    best_model = grid_search.best_estimator_
    print(f"Mejores hiperparámetros para {reg_name}: {grid_search.best_params_}")

    # Predicción en el conjunto de prueba
    y_pred = best_model.predict(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')  # Promedio ponderado para multiclase
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-Score (weighted): {f1:.4f}")

    # Mostrar clasificación detallada
    print("Reporte de clasificación:")
    print(classification_report(y_test, y_pred))

    # Importancia de las variables (coeficientes)
    if hasattr(best_model, 'coef_'):
        print("Importancia de las variables (coeficientes absolutos):")
        importance = pd.DataFrame({
            'Variable': X.columns,
            'Coeficiente': abs(best_model.coef_).mean(axis=0)  # Media de los coeficientes para multiclase
        }).sort_values(by='Coeficiente', ascending=False)
        print(importance.head(10))  # Mostrar las 10 más importantes
        # Exportar importancias a CSV
        importance.to_csv(f"100_variable_importance_{reg_name.replace(' ', '_')}.csv", index=False)
        print(f"Importancias exportadas a 'variable_importance_{reg_name.replace(' ', '_')}.csv'")

print("\nOptimización y evaluación completadas.")



Optimizando modelo con regularización: L2 (Ridge)
Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\usuario\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Mejores hiperparámetros para L2 (Ridge): {'C': 0.1}
Accuracy: 0.9810
F1-Score (weighted): 0.9809
Reporte de clasificación:
              precision    recall  f1-score   support

         1.0       0.99      1.00      1.00       368
         2.0       1.00      1.00      1.00       322
         3.0       1.00      0.99      1.00       296
         4.0       0.95      0.95      0.95       386
         5.0       0.96      0.96      0.96       412
         6.0       0.99      1.00      1.00       422

    accuracy                           0.98      2206
   macro avg       0.98      0.98      0.98      2206
weighted avg       0.98      0.98      0.98      2206

Importancia de las variables (coeficientes absolutos):
                                             Variable  Coeficiente
21  body_gyro_x__agg_autocorrelation__f_agg_"mean"...     0.299714
92                total_acc_y__number_crossing_m__m_0     0.296229
1   body_acc_x__time_reversal_asymmetry_statistic_...     0.224568
6    body_a

C:\Users\usuario\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\usuario\miniconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Mejores hiperparámetros para L1 (Lasso): {'C': 1}
Accuracy: 0.9814
F1-Score (weighted): 0.9814
Reporte de clasificación:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       368
         2.0       0.99      1.00      1.00       322
         3.0       1.00      0.99      1.00       296
         4.0       0.95      0.95      0.95       386
         5.0       0.96      0.95      0.96       412
         6.0       1.00      1.00      1.00       422

    accuracy                           0.98      2206
   macro avg       0.98      0.98      0.98      2206
weighted avg       0.98      0.98      0.98      2206

Importancia de las variables (coeficientes absolutos):
                                             Variable  Coeficiente
92                total_acc_y__number_crossing_m__m_0     0.409791
26  body_gyro_x__fft_coefficient__attr_"abs"__coeff_9     0.405229
21  body_gyro_x__agg_autocorrelation__f_agg_"mean"...     0.385587
24  body_gyro

C:\Users\usuario\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Mejores hiperparámetros para Elastic-Net: {'C': 0.1, 'l1_ratio': 0.1}
Accuracy: 0.9801
F1-Score (weighted): 0.9800
Reporte de clasificación:
              precision    recall  f1-score   support

         1.0       0.99      1.00      1.00       368
         2.0       0.99      1.00      1.00       322
         3.0       1.00      0.99      0.99       296
         4.0       0.95      0.95      0.95       386
         5.0       0.96      0.95      0.95       412
         6.0       1.00      1.00      1.00       422

    accuracy                           0.98      2206
   macro avg       0.98      0.98      0.98      2206
weighted avg       0.98      0.98      0.98      2206

Importancia de las variables (coeficientes absolutos):
                                             Variable  Coeficiente
21  body_gyro_x__agg_autocorrelation__f_agg_"mean"...     0.288441
92                total_acc_y__number_crossing_m__m_0     0.285030
1   body_acc_x__time_reversal_asymmetry_statistic_...     0.

In [40]:
import pandas as pd

# Cargar el nuevo dataset donde solo están las variables X
new_data = pd.read_csv("selected_features_rfe_test.csv")  # Reemplaza con el nombre de tu archivo
data_var_100 = pd.read_csv("100_variable_importance_Elastic-Net.csv")

# Filtrar solo las columnas seleccionadas
X_new = new_data[X.columns]

# Manejar valores faltantes (NaN) imputando con la media
X_new_imputed = imputer.transform(X_new)

# Escalar las características
X_new_scaled = scaler.transform(X_new_imputed)

# Realizar las predicciones
predictions = best_model.predict(X_new_scaled)

# Crear un DataFrame con las predicciones
results = pd.DataFrame({
    'ID': new_data.index+1,  # Asumimos que el índice es el identificador
    'Prediction': predictions
})

# Exportar el resultado a un archivo CSV
results.to_csv("predictions_kaggle_100var.csv", index=False)
print("Predicciones exportadas a 'predictions_kaggle_100var.csv'.")


Predicciones exportadas a 'predictions_kaggle_100var.csv'.


In [41]:
import joblib

# Guardar el modelo entrenado
joblib.dump(best_model, 'modelo_regresion_logistica_100V.pkl')
print("Modelo guardado en 'modelo_regresion_logistica.pkl'")

Modelo guardado en 'modelo_regresion_logistica.pkl'


In [42]:
X.columns

Index(['body_acc_x__count_above_mean',
       'body_acc_x__time_reversal_asymmetry_statistic__lag_3',
       'body_acc_x__c3__lag_1',
       'body_acc_x__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'body_acc_x__spkt_welch_density__coeff_2',
       'body_acc_x__ar_coefficient__coeff_2__k_10',
       'body_acc_x__fft_coefficient__attr_"abs"__coeff_4',
       'body_acc_x__fft_coefficient__attr_"angle"__coeff_1',
       'body_acc_x__friedrich_coefficients__coeff_2__m_3__r_30',
       'body_acc_x__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"',
       'body_acc_x__lempel_ziv_complexity__bins_3',
       'body_acc_y__number_cwt_peaks__n_5',
       'body_acc_y__fft_coefficient__attr_"abs"__coeff_4',
       'body_acc_z__quantile__q_0.4',
       'body_acc_z__partial_autocorrelation__lag_2',
       'body_acc_z__number_peaks__n_3', 'body_acc_z__number_peaks__n_10',
       'body_acc_z__index_mass_quantile__q_0.2',
       'body_acc_z__fft_coefficient__attr_"imag"__coeff_1',


In [44]:
df = pd.DataFrame(X.columns, columns=['Nombres'])
# Exportar el DataFrame a un archivo CSV
df.to_csv('nombres_variables_100_modelo_regresion_logistica.csv', index=True)
print("Los nombres se han exportado a nombres.csv.")

Los nombres se han exportado a nombres.csv.


SVM

In [45]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import joblib

# Cargar el dataset seleccionado
selected_features = pd.read_csv("selected_features_rfe_optimized_100.csv")

# Separar las características (X) y la variable objetivo (y)
X = selected_features.drop(columns=['label'])  # Características
y = selected_features['label']  # Variable objetivo

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# **Reducción de dimensionalidad con PCA**
pca = PCA(n_components=0.95)  # Mantener 30 componentes principales
X_pca = pca.fit_transform(X_scaled)

# Guardar el PCA para futuras predicciones
joblib.dump(pca, 'pca_transform_100v.pkl')
print("PCA guardado en 'pca_transform_100v.pkl'.")

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42, stratify=y)

# Definir los parámetros para la búsqueda de hiperparámetros
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],  # Tipos de kernel
    'C': [0.01, 0.1, 1, 10, 100],  # Regularización
    'gamma': ['scale', 'auto'],  # Coeficiente del kernel
    'degree': [2, 3, 4]  # Solo aplicable para kernel 'poly'
}

# Crear el modelo base
svm = SVC(decision_function_shape='ovo', random_state=42)

# Configurar GridSearchCV con validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='f1_weighted', cv=cv, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Mejor modelo
best_svm = grid_search.best_estimator_
print(f"Mejores hiperparámetros para SVM: {grid_search.best_params_}")

# Predicción en el conjunto de prueba
y_pred = best_svm.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')  # Promedio ponderado para multiclase
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score (weighted): {f1:.4f}")

# Mostrar clasificación detallada
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

# Exportar el modelo optimizado
joblib.dump(best_svm, 'svm_model_optimized_100v.pkl')
print("Modelo SVM optimizado guardado en 'svm_model_optimized.pkl'.")


PCA guardado en 'pca_transform_100v.pkl'.
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Mejores hiperparámetros para SVM: {'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 0.9782
F1-Score (weighted): 0.9782
Reporte de clasificación:
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       368
         2.0       0.99      1.00      1.00       322
         3.0       1.00      0.99      1.00       296
         4.0       0.94      0.95      0.94       386
         5.0       0.96      0.94      0.95       412
         6.0       0.99      1.00      0.99       422

    accuracy                           0.98      2206
   macro avg       0.98      0.98      0.98      2206
weighted avg       0.98      0.98      0.98      2206

Modelo SVM optimizado guardado en 'svm_model_optimized.pkl'.


In [49]:
import pandas as pd
import joblib
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Cargar el modelo SVM y el PCA guardados
svm_model = joblib.load('svm_model_optimized_100v.pkl')
pca = joblib.load('pca_transform_100v.pkl')
print("Modelo SVM y PCA cargados con éxito.")

# Cargar el nuevo dataset
new_data = pd.read_csv("selected_features_rfe_test.csv")  # Reemplaza con el nombre correcto de tu archivo
print("Nuevo dataset cargado.")

# Asegurarse de que las columnas seleccionadas estén presentes en el nuevo dataset

# Seleccionar solo las columnas necesarias
X_new = new_data[X.columns]

# Manejar valores faltantes en el nuevo dataset
imputer = SimpleImputer(strategy='mean')
X_new_imputed = imputer.fit_transform(X_new)

# Escalar los datos
scaler = StandardScaler()
X_new_scaled = scaler.fit_transform(X_new_imputed)

# Aplicar PCA al nuevo dataset
X_new_pca = pca.transform(X_new_scaled)

# Hacer predicciones con el modelo SVM cargado
predictions = svm_model.predict(X_new_pca)

# Crear un DataFrame con las predicciones
results = pd.DataFrame({
    'ID': new_data.index + 1,  # Asumimos que el índice es el identificador
    'Prediction': predictions
})

# Exportar las predicciones a un archivo CSV
results.to_csv("predictions_kaggle_svm.csv", index=False)
print("Predicciones exportadas a 'predictions_kaggle_svm_100v.csv'.")


Modelo SVM y PCA cargados con éxito.
Nuevo dataset cargado.
Predicciones exportadas a 'predictions_kaggle_svm_100v.csv'.
